In [1]:
from my_secrets import MONGODB_USERNAME, MONGODB_PASSWORD
from pymongo import MongoClient

# Requires the PyMongo package.
# https://api.mongodb.com/python/current

cluster_uri = "matsci-opt-benchmarks.ehu7qrh"
client = MongoClient(f"mongodb+srv://{MONGODB_USERNAME}:{MONGODB_PASSWORD}@{cluster_uri}.mongodb.net/?retryWrites=true&w=majority")

database_name = "crabnet-hyperparameter"
collection_name = "sobol"

In [2]:
from tqdm.notebook import tqdm
db = client[database_name]
collection = db[collection_name]
# results = collection.aggregate([{"$match": {"fba": {"$exists": True}}}])
filter = {"hardware": {"$exists": True}, "scores": {"$exists": True}}
num_docs = collection.count_documents(filter)
cursor = collection.find(filter)
results = []
for result in tqdm(cursor, total=num_docs):
    results.append(result)

  0%|          | 0/173219 [00:00<?, ?it/s]

In [3]:
import pandas as pd
import json
df = pd.DataFrame(results).set_index("_id")
df["scores"] = df["scores"].apply(json.dumps)
df.to_csv(f"../../data/external/{database_name}-{collection_name}.csv")
df.columns

Index(['N', 'alpha', 'd_model', 'dim_feedforward', 'dropout', 'emb_scaler',
       'eps', 'epochs_step', 'fudge', 'heads', 'k', 'lr', 'pe_resolution',
       'ple_resolution', 'pos_scaler', 'weight_decay', 'batch_size',
       'out_hidden4', 'betas1', 'betas2', 'bias', 'train_frac', 'criterion',
       'elem_prop', 'hardware', 'sample_seed', 'scores', 'model_size',
       'runtime', 'session_id', 'timestamp', 'date', 'sobol_seed',
       'num_sobol_samples', 'num_repeats'],
      dtype='object')

In [4]:
df["runtime"].mean() / 60

3.2171862053427316